In [1]:
import numpy as np
import pandas as pd

In [8]:
from sklearn.preprocessing import LabelEncoder

In [2]:
data = pd.read_csv('data.csv')

In [9]:
le = LabelEncoder()

In [11]:
cat_features = ['zip_code', 'channel', 'offer']
for column in cat_features:
    le.fit(data[column])
    data[column] = le.transform(data[column])

In [12]:
data

,recency,history,used_discount,used_bogo,zip_code,is_referral,channel,offer,conversion
0,10,142.44,1,0,1,0,1,0,0
1,6,329.08,1,1,0,1,2,2,0
2,7,180.65,0,1,1,1,2,0,0
3,9,675.83,1,0,0,1,2,1,0
4,2,45.34,1,0,2,0,2,0,0
...,...,...,...,...,...,...,...,...,...
63995,10,105.54,1,0,2,0,2,1,0
63996,5,38.91,0,1,2,1,1,1,0
63997,6,29.99,1,0,2,1,1,1,0
63998,1,552.94,1,0,1,1,0,0,0


In [6]:
data_treatment = data[data.offer != 'No Offer']
data_control = data[data.offer == 'No Offer']

In [10]:
data

,recency,history,used_discount,used_bogo,zip_code,is_referral,channel,offer,conversion
0,10,142.44,1,0,Surburban,0,Phone,Buy One Get One,0
1,6,329.08,1,1,Rural,1,Web,No Offer,0
2,7,180.65,0,1,Surburban,1,Web,Buy One Get One,0
3,9,675.83,1,0,Rural,1,Web,Discount,0
4,2,45.34,1,0,Urban,0,Web,Buy One Get One,0
...,...,...,...,...,...,...,...,...,...
63995,10,105.54,1,0,Urban,0,Web,Discount,0
63996,5,38.91,0,1,Urban,1,Phone,Discount,0
63997,6,29.99,1,0,Urban,1,Phone,Discount,0
63998,1,552.94,1,0,Surburban,1,Multichannel,Buy One Get One,0


In [5]:
data[data.offer == 'No Offer']

,recency,history,used_discount,used_bogo,zip_code,is_referral,channel,offer,conversion
1,6,329.08,1,1,Rural,1,Web,No Offer,0
14,4,241.42,0,1,Rural,1,Multichannel,No Offer,0
15,3,58.13,1,0,Urban,1,Web,No Offer,1
20,9,29.99,0,1,Surburban,1,Phone,No Offer,0
23,2,29.99,0,1,Urban,1,Phone,No Offer,0
...,...,...,...,...,...,...,...,...,...
63980,3,487.10,0,1,Surburban,1,Phone,No Offer,0
63981,4,125.53,0,1,Rural,1,Phone,No Offer,0
63983,2,83.03,0,1,Urban,0,Phone,No Offer,0
63987,1,79.70,1,0,Surburban,1,Web,No Offer,0
